In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [2]:
df_UBC = pd.read_csv("Gbif_UBC.csv", delimiter="\t", low_memory=False)
df_UBC = df_UBC.head(50)

In [3]:
geometry = [Point(xy) for xy in zip (df_UBC['decimalLongitude'], df_UBC['decimalLatitude'])]

In [4]:
geo_df = gpd.GeoDataFrame(df_UBC, geometry = geometry, crs = {'init': 'epsg:4326'})

In [24]:
geo_df.to_file("leaflet/UBC.geojson", driver="GeoJSON")